# Classification of text documents using sparse features

This is an example showing how scikit-learn can be used to classify documents
by topics using a bag-of-words approach. This example uses a scipy.sparse
matrix to store the features and demonstrates various classifiers that can
efficiently handle sparse matrices.


In [1]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# Updated by: Kisti
# Finalized by: Kamil Veli TORAMAN <kvtoraman@kaist.ac.kr>
# 23 August 2017
# License: BSD 3 clause

from __future__ import print_function
import pandas as pd
import logging
import numpy as np
import nltk.stem
from optparse import OptionParser
import sys, copy
from time import time
from random import randint
#import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import fetch_rcv1

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
def apk_per_class(clf, actual, predicted, k=5):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items per each class
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : multi list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    class_score = {}
    micro_correct = 0.0
    length = 0
    global ee_correct,ee_tested
    global cumulative_class_score
    ee_correct = 0
    ee_tested = 0
    if len(actual) == len(predicted):		

        for	i in range(len(actual)):
            if actual[i] not in class_score:
                class_score[actual[i]] = [0.0, 0.0] # correct_score, length
            if actual[i] not in cumulative_class_score:
                cumulative_class_score[actual[i]] = [0.0, 0.0] # correct_score, length
                
            well_classified = False
            for pred in predicted[i]:
                if actual[i] in pred:
                    #if(randint(0, 500) == 50):
                    #	print("act: -" , actual[i], "-pred:", pred,":")
                    class_score[actual[i]][0] += 1.0
                    micro_correct += 1.0
                    well_classified = True
            if(type(actual[i]) is list):
                print("!! actual[",i,"] is ",actual[i])
            if(actual[i] == CODE):
                print(well_classified , "docID:",i,"prediction of ",CODE," was:",predicted[i])
                fail_list_for_code.append(i)
                #print(test_dtm[i].toarray()[0])
                #for j,word_rat in enumerate(test_dtm[i].toarray()[0]):
                #	if(word_rat>0.2):
                #		print(word_rat)
            class_score[actual[i]][1] += 1.0
            length+=1

    avg_acc = 0.0
    
    for cl in class_score.keys():
        avg = class_score[cl][0]/class_score[cl][1]
        if(avg<0.6 and count[cl]-class_score[cl][1] > class_score[cl][1]):
            print("!Low precision :! #Correct:", class_score[cl][0], "#Tested:", class_score[cl][1],"#Train",count[cl]-class_score[cl][1])
            fail_list.append(cl)
        print ("\t", cl, "Acc.:", avg, "Correct:", class_score[cl][0], "Tested:", class_score[cl][1],"#Train",count[cl]-class_score[cl][1])
        if cl[0] =='E':
            ee_correct += class_score[cl][0]
            ee_tested += class_score[cl][1]
        avg_acc +=avg
        cumulative_class_score[cl][0] += class_score[cl][0]
        cumulative_class_score[cl][1] += class_score[cl][1]
        

    print ('Total Test Examples', length, "\nMicro Acc.(item level)", micro_correct/length)
    print('Average of EE',ee_correct/ee_tested) 
    global cumulative_micro_avg
    cumulative_micro_avg += micro_correct/length
    return avg_acc/len(class_score)

###############################################################################
# Benchmark classifiers
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()

    # Top 1 
    pred = clf.predict(X_test)    
    probs = clf.predict_proba(X_test)

    for topk in range(5,6):
        best_n_label = transform_label(clf, probs, topk)

        test_time = time() - t0
        print("test time:  %0.3fs" % test_time)

        pred = best_n_label
        print ("Top-", topk)
        print ("Macro Acc.(class level)", apk_per_class(clf, y_test, best_n_label, topk), "\n\n")

In [3]:
def transform_label(clf, prob, topk):
    rst_arr = np.empty( (len(prob), topk), dtype=object) 
    for i in range(len(prob)):
        s_items = np.argsort(prob[i])[-topk:]

        for j in range(len(s_items)):
            rst_arr[i][j] = clf.classes_[s_items[j]]
    return rst_arr


In [7]:

###############################################################################
# Load some categories from the training set
CODE = "LC04" #the CODE we observe if we debug
categories =  [x for x in open('KSCC_sample_data_170206_Codelist.dat','r').read().split('\n') if len(x) > 0]

print("Loading NSCC  dataset for categories:")
print(categories if categories else "all")
print("Category size",len(categories))

#data_train = open('KSSC_sample_data_170206_Train.dat').readlines()
#data_test = open('KSSC_sample_data_170206_Test.dat').readlines()
data_train = open('rev_utf8_train.dat').readlines()
#all_data = open('rev_reserved_new_data_all.dat').readlines()
all_data = open('kkma_koreanonly_withsentencebreaker_2cols.dat').readlines()
#data_test = open('rev_utf8_test.dat').readlines()
data_test = open("rev_utf8_test.dat").readlines()
ENCODING = 'utf-8'
data_train_data, data_test_data = [], []
y_train, y_test = [], []
all_x = []
count = {}
all_y = []

#read all data    
for i,line in enumerate(all_data):
    items = line.split('\t')
    if len(items) == 2:
        if items[0][0:2] != 'EE':
            continue
        items[1] = items[1].split('%%') #name + ' %% ' + kor_kywd + ' %% ' + goal + ' %% ' + abstract + ' %% ' + efct
        occur_many_times = ""
        occur_many_times += items[1][0]*1 + " "
        occur_many_times += items[1][1]*1 + " "
        items[1] =occur_many_times + ' '.join(items[1][2:])
        all_x.append(items[1].decode(ENCODING, 'ignore'))
        #if(items[0] == CODE):
        #    print(line)
        all_y.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
    else:
        print("ERROR in trai n",i,len(items))
"""      
for i,line in enumerate(data_train):
    items = line.split('\t')
    if len(items) == 2:
        data_train_data.append(items[1].decode(ENCODING, 'ignore'))
        y_train.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
    else:
        print("ERROR in train",i,len(items))
for i,line in enumerate(data_test):
    items = line.split('\t')
    if len(items) == 2:
        data_test_data.append(items[1].decode(ENCODING, 'ignore'))
        y_test.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
    else:
        print("ERROR in test",i,len(items))
"""

5

Loading NSCC  dataset for categories:
['ED10', 'ED11', 'EI02', 'EI03', 'EI06', 'EI07', 'EI05', 'EI08', 'EH06', 'EF99', 'EB01', 'EA09', 'EE11', 'EE10', 'EE13', 'EE12', 'EI99', 'EE14', 'EA04', 'EA05', 'ED07', 'ED06', 'ED05', 'ED04', 'ED03', 'ED01', 'EE99', 'ED08', 'EA02', 'EH10', 'EI11', 'EI12', 'EA14', 'EA11', 'EA10', 'EA13', 'EA07', 'EF05', 'EF06', 'ED99', 'EE08', 'EE09', 'EE06', 'EE07', 'EE04', 'EE05', 'EE02', 'EE03', 'EE01', 'SB99', 'ND07', 'OA04', 'LC06', 'SI04', 'SH07']
Category size 55


5

In [8]:
guess = 80000*[""]
#divides data to train&test with 9:1 ratio
def divide_data(rs):    
    global data_train_data,data_test_data,y_train,y_test,guess
    data_train_data,data_test_data,y_train,y_test = train_test_split(all_x,all_y,random_state =rs, train_size = 0.9)
    print (len(data_train_data), len(data_test_data))
    print('data loaded')
    guess = 80000*[""]
    #data_train_data,data_test_data = data_test_data,data_train_data
    #y_test,y_train=y_train,y_test
    #data_train_data = data_test_data
    #y_train = y_test


In [9]:
#vectorize training data using tfidf vectorizer
vectorizer,X_test,X_train = 0,0,0
def vectorize():
    print("Extracting features from the training data using a sparse vectorizer")
    my_stop_words = [np.unicode(x.strip(), 'utf-8','ignore') for x in open('kor_stop_word.txt', 'r').read().split('\n')]
    global vectorizer,X_test,X_train

    vectorizer = TfidfVectorizer(max_df=0.5,stop_words=my_stop_words,max_features = 100000,
                                 min_df=3,ngram_range=(1,1))
    #vectorizer = StemmedTfidfVectorizer(stop_words=my_stop_words,max_df=0.5,max_features = 50000,min_df=3)    
    X_train = vectorizer.fit_transform(data_train_data)
    print("n_samples: %d, n_features: %d" % X_train.shape)
    print("Extracting features from the test data using the same vectorizer")
    X_test = vectorizer.transform(data_test_data)
    print("n_samples: %d, n_features: %d" % X_test.shape)
    # mapping from integer feature name to original token string

In [10]:
#results = []
# Train SGD model
accumulated_fail_list = {}
TRIAL_SIZE = 3
RANDOM_STATE_START = 5
cumulative_micro_avg = 0
ee_correct = 0
ee_tested = 0
cumulative_class_score = {}
CODE = 'EE14' #code you want to investigate, doesn't affect the result
#try the code with different train/test splts
for t,trial in enumerate(range(RANDOM_STATE_START,RANDOM_STATE_START + TRIAL_SIZE)):
    print("Trial:",t,"R_s",trial)
    divide_data(trial)
    vectorize()
    feature_names = vectorizer.get_feature_names()
    feature_names = np.asarray(feature_names)
    fail_list = []
    fail_list_for_code = []
    cumulative_class_score = {}
    suggested_n_iter = np.ceil(10**6/len(data_train_data))
    clf = SGDClassifier(loss='log', alpha=.0001, n_iter=50, penalty="l2")
    benchmark(clf)
    for x in fail_list:
        if not accumulated_fail_list.has_key(x):
            accumulated_fail_list[x] = 0
        accumulated_fail_list[x] +=1
    #print(fail_list)
    for x in accumulated_fail_list:
        if accumulated_fail_list[x] == TRIAL_SIZE:
            print(x,accumulated_fail_list[x])
print('Cumulative',cumulative_micro_avg/TRIAL_SIZE)

Trial: 0 R_s 5
8832 982
data loaded
Extracting features from the training data using a sparse vectorizer
n_samples: 8832, n_features: 25473
Extracting features from the test data using the same vectorizer
n_samples: 982, n_features: 25473
________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
train time: 4.109s
test time:  0.014s
Top- 5
True docID: 22 prediction of  EE14  was: ['EE05' 'EE11' 'EE14' 'EE99' 'EE02']
False docID: 33 prediction of  EE14  was: ['EE04' 'EE99' 'EE01' 'EE09' 'EE06']
False docID: 104 prediction of  EE14  was: ['EE02' 'EE08' 'EE10' 'EE09' 'EE03']
False docID: 228 prediction of  EE14  was: ['EE13' 'EE99' 'EE06' 'EE04' 'EE09']
True docID: 

In [11]:
#this block prints the statistics(info about each category)
statistics = open('statistics_2ndtest.txt','w')
statistics.write('\t'.join(['CODE','ALL','TRAIN','TEST','CORRECT','correct%'])+'\n')

for x in count:
    a_row = [x,count[x],count[x]-cumulative_class_score[x][1],cumulative_class_score[x][1],cumulative_class_score[x][0],cumulative_class_score[x][0]/cumulative_class_score[x][1]]#code,train,tested,correct
    for i in range(len(a_row)):
        a_row[i] = str(a_row[i])
    statistics.write('\t'.join(a_row)+'\n')
statistics.close()

In [12]:
#which categories failed most
for x in accumulated_fail_list:
        if accumulated_fail_list[x] >= TRIAL_SIZE -2:
            print(x,accumulated_fail_list[x])

EE14 3


In [13]:
#which categories failed most 2
for x in cumulative_class_score:
    if(x[0] == "E"):
        if(cumulative_class_score[x][0]/cumulative_class_score[x][1]<0.7):
            print(x,cumulative_class_score[x],count[x]*TRIAL_SIZE)

EE14 [7.0, 13.0] 387


In [36]:
out = file("features.txt","w")

for x in feature_names:
    out.write(x.encode('utf-8','ignore')+"\n")
out.close()

In [37]:
#get the index for specific code
#class index of EE02 is 34
#class index of LC06 is 94
#class index of EE99 is 47
#class index of EA10 is 9 
#class index of EF05 is 50 
#
CODE_INDEX = clf.classes_.tolist().index('EE14')
CODE_INDEX

13

In [38]:
#deceiving category? EE02?
#Powerful features of EE02, the biggest category
INDEX_BLACK = clf.classes_.tolist().index('EE02')
black_list = []
print(clf.classes_[INDEX_BLACK])
for i,x in enumerate(vectorizer.get_feature_names()):
    if(3>clf.coef_[INDEX_BLACK][i]>1):
        print(clf.coef_[INDEX_BLACK][i],x)
        black_list.append(x)


EE02
1.2325249747 개인정보
1.53592121446 거래
1.58752524746 게임
1.6019800809 고객
1.06334322866 공유
1.3130082847 과정
2.09353514583 관리
1.22406217274 관리자
1.21006990922 교사
1.2828301061 구매
1.10887280068 글로벌
2.32901317571 기능
1.19942699853 기업
1.16084359737 내역
1.08898348963 다운로드
1.11972726869 단위
1.0697982454 대화
1.03332397988 데이타
1.37578268902 데이터
1.67501780483 디드
1.27686718126 레이션
1.42167953134 렌더링
1.2567196695 로그
1.07007469048 마이그레이션
1.49753887385 마케팅
1.36835061126 마켓
1.01865588702 매쉬업
2.22634493471 모바일
1.00687164547 모바일앱
1.29037957728 문서
1.53119627125 번역
1.70143294684 보유
1.25281211344 복합무기체계
1.05514928308 비만
1.02627496833 사운드
1.3013945151 서버
1.01473475988 성과관리
2.34078471461 소프트
2.42094858084 소프트웨어
1.19211562874 스트리밍
1.25643794474 시뮬레이션
2.00371658427 실시간
1.96938857832 엔진
1.05957562027 여행
1.63020720646 온라인
1.00890399597 운영
1.5303715275 운영체제
2.61685225179 웨어
1.75043514342 웹기반
1.15862432564 웹서비스
1.03616129524 위젯
1.26197142059 융합기술고도화
1.28375374837 음원
1.17545248362 이미지
1.65287924891 임베디드
1.44318784964 임상
1

In [39]:
white_list = []
for i,x in enumerate(vectorizer.get_feature_names()):
    if(clf.coef_[CODE_INDEX][i]>1):
        print(clf.coef_[CODE_INDEX][i],x)
        white_list.append(x)
    

1.06732238452 감시
1.86155732735 개량
3.48116191995 국방
1.52780349699 레이더
1.10094687134 모델
1.92851975951 모의
1.20963622893 무선메쉬백
1.12248950871 방해
1.71841679384 분석모델
1.04816316201 비기상
1.69777584919 성능개량
1.20562308521 연합
2.9658528845 전술
1.07343715189 전투
1.49286358724 지상
1.11350868593 체계
1.42480281347 통신미들웨어
1.08714627132 특화연구센터
1.39324437543 특화연구실
1.08566165472 함정
1.81951303371 합동


In [40]:

print(y_test[fail_list_for_code[0]],data_test_data[fail_list_for_code[0]])
fail_data = []
for x in fail_list_for_code:
    fail_data.append(data_test_data[x])

EE14 국방 전술 전술통신 통신 기술 연구   국방 전술 전술데이터링크 데이터 링크 데이터링크 합동 완성 차기 차기위성 위성   국방 전술 전술데이터링크 데이터 링크 구축 전술통신 통신 네트워크 핵심 기술 개발 분야 이론 실무 겸비 전문 인력 양성   국방 전술 데이터 링크 인지 기반 주파수 관리 방어 설계 안전 그룹 통신 보안 전송 기술 전투 전투무선망 무선망 효과 효과적인 적인 무선 전송제어 제어 재 재밍 밍 환경 신뢰성 향상 재 재밍 극복 망 생존 유지 체계 전술데이터링크 그룹간 간 네트워크 운용 위성 지상 지상망 연동 라 라우팅 우 팅 전략 보장 데이터링크 간 협업 규칙 매칭 시스템 간섭 영향 최소화 그룹협력통신 협력 지원 전술네트워크 자원 자원관리 프로 프로토콜 토 콜 협력기반 확보 스마트   산업 기여도 국방 국방분야 분야 기대 파급 파급효과 효과 전장 전장공간 공간 정보 정보우위 우위 확보 실시간 교전 교전능력 능력 향상 핵심 핵심요소 요소 기술 육 해 공군 합동 합동작전 작전 향후 센서 무기 타격 타격체계 체계 적용 군수 군수물자 물자 수출 기여 수 해외 수입 막대 대체 취득 추후 우주 국방시대 시대 요구 통신 네트워크 기술요소 응용 국가 경쟁력 증대 이바지 가능 차세대 전술 전술데이터링크 데이터 링크 개발 한국군 무기체계 첨단 가속 민간 공공 공공분야 전자 관련 핵심기술 군수산업 부가 부가가치 가치 이동 이동통신 4 5 등 상용 원천 국제 표준화 주도 재난 경보 구조 구조체계 적용가능 상황 상황공유 공유 자원 절약 인명 인명피해 피해 최소화 데이터링크 요소기술 항공기 헬리콥터 인공 인공위성 위성 다양 기업 산업화 창출 선진국 의존도 낮춤 경제적 민간분야 기대효과 현재 미군 변경 정도 기술력 사용 원천기술료 기술료 지급 상태 독자적 이용 통해 뿐 국방연구개발비 연구 개발비 절감 부족 국방비 기초 운용 인터넷 인터넷망 망 구성 중추적 역할 담당 동시 통신기술 국가경쟁력 독자 독자적인 적인 동남아 지역 군 천문학 국가적 예상 기술인 앞 한국 주도적

In [46]:
v = vectorizer.transform(fail_data)
v_array = v.toarray()
important_word = 80000*[0]
buffer = 80000*[0]
for i,doc in enumerate(fail_data):
    print(doc)
    #print(v.toarray()[i])
    #rev_list =  reversed(np.argsort(v.toarray()[i]))
    for j in range(len(v_array[i])):
        if(v_array[i][j]>0.0):
     #       print(index,feature_names[index])
            buffer[j] = 1
        
    for j in range(len(buffer)):
        if buffer[j] > 0:
            important_word[j] += 1
            buffer[j] = 0
            


국방 전술 전술통신 통신 기술 연구   국방 전술 전술데이터링크 데이터 링크 데이터링크 합동 완성 차기 차기위성 위성   국방 전술 전술데이터링크 데이터 링크 구축 전술통신 통신 네트워크 핵심 기술 개발 분야 이론 실무 겸비 전문 인력 양성   국방 전술 데이터 링크 인지 기반 주파수 관리 방어 설계 안전 그룹 통신 보안 전송 기술 전투 전투무선망 무선망 효과 효과적인 적인 무선 전송제어 제어 재 재밍 밍 환경 신뢰성 향상 재 재밍 극복 망 생존 유지 체계 전술데이터링크 그룹간 간 네트워크 운용 위성 지상 지상망 연동 라 라우팅 우 팅 전략 보장 데이터링크 간 협업 규칙 매칭 시스템 간섭 영향 최소화 그룹협력통신 협력 지원 전술네트워크 자원 자원관리 프로 프로토콜 토 콜 협력기반 확보 스마트   산업 기여도 국방 국방분야 분야 기대 파급 파급효과 효과 전장 전장공간 공간 정보 정보우위 우위 확보 실시간 교전 교전능력 능력 향상 핵심 핵심요소 요소 기술 육 해 공군 합동 합동작전 작전 향후 센서 무기 타격 타격체계 체계 적용 군수 군수물자 물자 수출 기여 수 해외 수입 막대 대체 취득 추후 우주 국방시대 시대 요구 통신 네트워크 기술요소 응용 국가 경쟁력 증대 이바지 가능 차세대 전술 전술데이터링크 데이터 링크 개발 한국군 무기체계 첨단 가속 민간 공공 공공분야 전자 관련 핵심기술 군수산업 부가 부가가치 가치 이동 이동통신 4 5 등 상용 원천 국제 표준화 주도 재난 경보 구조 구조체계 적용가능 상황 상황공유 공유 자원 절약 인명 인명피해 피해 최소화 데이터링크 요소기술 항공기 헬리콥터 인공 인공위성 위성 다양 기업 산업화 창출 선진국 의존도 낮춤 경제적 민간분야 기대효과 현재 미군 변경 정도 기술력 사용 원천기술료 기술료 지급 상태 독자적 이용 통해 뿐 국방연구개발비 연구 개발비 절감 부족 국방비 기초 운용 인터넷 인터넷망 망 구성 중추적 역할 담당 동시 통신기술 국가경쟁력 독자 독자적인 적인 동남아 지역 군 천문학 국가적 예상 기술인 앞 한국 주도적 기술적 

In [44]:
#words that appear frequently in our low-quality code
for i,word_id in enumerate(important_word):
    if word_id > 4:
        print(word_id,feature_names[i])

7 구조
8 구축
5 국방
7 네트워크
6 다양
5 데이터
5 무선
5 보안
6 분야
6 사용
8 설계
7 수행
5 실시간
6 응용
8 이용
8 적용
7 전송
5 제어
6 지원
5 채널
6 처리
7 통신
7 확보
5 환경
5 효율적


In [47]:
test_dtm = vectorizer.transform(data_test_data)
train_dtm = vectorizer.transform(data_train_data)
table = pd.DataFrame(test_dtm.toarray())
s = pd.Series([45])
problems = pd.DataFrame()
f_names = pd.Series(vectorizer.get_feature_names())
for i,v in enumerate(s):
    problems = problems.append(table.loc[s[i], table.loc[s[i]]>0],ignore_index = True)
vocab = problems.T
vocab['meaning']= f_names[vocab.index]
fn_list = f_names.tolist()
#vocab['meaning']= vocab['meaning'].apply(lambda x: x.encode(encoding='utf-8',errors="ignore"))
vocab = vocab[vocab[0]>0.1]
#display(vocab)

for i,x in enumerate(vocab['meaning']):
    if x in black_list:
        print("Black L found",i,x)
        print(x)

for i,x in enumerate(vocab['meaning']):
    if x in white_list:
        print("White L found",i,x)
        

Black L found 7 소프트
소프트
Black L found 8 소프트웨어
소프트웨어


In [48]:
#english stemmer, not used . It doesn't improve the accuracy
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer,self).build_analyzer()
        return lambda doc: (
        english_stemmer.stem(w) for w in analyzer(doc))

In [49]:
table = pd.DataFrame(train_dtm.toarray())
s = pd.Series([59,60,61,62,63,64,65])
problems = pd.DataFrame()
f_names = pd.Series(vectorizer.get_feature_names())
for i in range(6):
    problems = problems.append(table.loc[s[i], table.loc[s[i]]>0],ignore_index = True)
vocab = problems.T
vocab['meaning']= f_names[vocab.index]
fn_list = f_names.tolist()
#vocab['meaning']= vocab['meaning'].apply(lambda x: x.encode('utf-8'))
#vocab